In [1]:
using LinearAlgebra
using Roots
using DataFrames
using CSV
using StaticArrays
using BenchmarkTools

In [2]:
const ϵ₀ = 1/8
const ΔV = 1/8
const Tₗ = 1/8
const Tᵣ = 1/8
const W = 1.0
const W° = 1/2
const L₁ = 160
const L₂ = 40

40

In [3]:
function FermiDirac(ϵ::Float64, μ::Float64, T::Float64)
    return 1/(1+exp((ϵ-μ)/T))
end

@benchmark $FermiDirac($ϵ₀, $0.0, $Tᵣ)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  5.043 ns … 14.419 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.622 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.608 ns ±  0.481 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                   █▃   ▄                                     
  ▄█▃▂▃▄▂▃▄▅▂▃▃▄▂▃▄██▃▂▅█▄▆▃▁▁▁▅▂▁▂▁▂▂▂▂▁▁▁▁▂▁▁▂▂▁▂▂▂▂▂▂▂▂▂▂ ▃
  5.04 ns        Histogram: frequency by time        6.92 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [4]:
@code_warntype FermiDirac(ϵ₀, 0.0, Tᵣ)

MethodInstance for FermiDirac(::Float64, ::Float64, ::Float64)
  from FermiDirac(ϵ::Float64, μ::Float64, T::Float64) in Main at In[3]:1
Arguments
  #self#::Core.Const(FermiDirac)
  ϵ::Float64
  μ::Float64
  T::Float64
Body::Float64
1 ─ %1 = (ϵ - μ)::Float64
│   %2 = (%1 / T)::Float64
│   %3 = Main.exp(%2)::Float64
│   %4 = (1 + %3)::Float64
│   %5 = (1 / %4)::Float64
└──      return %5



In [5]:
function ExponentialRate(L::Int64, C::Float64)
    f(x) = x*(1.0-x^L)/(1.0-x+1e-12)-C
    return find_zero(f, (1.0, C), A42())
end

Δϵ = 2*W°/(L₁-1)
a = (W-W°)/Δϵ
b = L₂÷2

@btime $ExponentialRate($b, $a)

  905.659 ns (0 allocations: 0 bytes)


1.1188004365129958

In [6]:
using LoopVectorization
using LazyArrays
using FillArrays

function BathSpectra(W::Float64, W°::Float64, L₁::Int64, L₂::Int64)
    Δϵ = 2.0*W°/(L₁-1)
    Φ = ExponentialRate(L₂÷2, (W-W°)/Δϵ)
    
    Range1 = 1:L₁÷2
    Range2 = 1:L₂÷2
    exponent = @. Φ^Range2
    aux = (W-W°)/(1-Φ^(L₂÷2))
    
    ϵ₁ = @. Δϵ*(Range1 - 0.5)
    ϵ₂ = @. W° + aux *(1.0 - exponent)
    
    ϵ = ApplyArray(vcat, reverse(-ϵ₂), reverse(-ϵ₁), ϵ₁, ϵ₂)

    γ₁ = Fill(Δϵ, L₁÷2)
    γ₂ = @. Δϵ*Φ^ϵ₂

    γ = ApplyArray(vcat, reverse(γ₂), reverse(γ₁), γ₁, γ₂)
    
    return ϵ, γ
end

@btime $BathSpectra($W, $W°, $L₁, $L₂)

  1.734 μs (6 allocations: 1.31 KiB)


([-1.0, -0.9406179182442861, -0.8875413553312501, -0.8401007562924605, -0.7976976632201611, -0.7597971657869831, -0.7259211534113499, -0.6956422839453924, -0.6685785928010373, -0.6443886745089908  …  0.6443886745089908, 0.6685785928010373, 0.6956422839453924, 0.7259211534113499, 0.7597971657869831, 0.7976976632201611, 0.8401007562924605, 0.8875413553312501, 0.9406179182442861, 1.0], [0.007036480732786138, 0.006989731130391348, 0.006948208600585231, 0.00691130397433998, 0.006878484016187124, 0.006849280993363852, 0.006823283859637566, 0.006800130771295097, 0.006779502707804832, 0.006761118013004732  …  0.006761118013004732, 0.006779502707804832, 0.006800130771295097, 0.006823283859637566, 0.006849280993363852, 0.006878484016187124, 0.00691130397433998, 0.006948208600585231, 0.006989731130391348, 0.007036480732786138])

In [7]:
Γ = LinRange(0.0, 0.5, 20)
ϵ, γ = BathSpectra(W, W°, L₁, L₂)

([-1.0, -0.9406179182442861, -0.8875413553312501, -0.8401007562924605, -0.7976976632201611, -0.7597971657869831, -0.7259211534113499, -0.6956422839453924, -0.6685785928010373, -0.6443886745089908  …  0.6443886745089908, 0.6685785928010373, 0.6956422839453924, 0.7259211534113499, 0.7597971657869831, 0.7976976632201611, 0.8401007562924605, 0.8875413553312501, 0.9406179182442861, 1.0], [0.007036480732786138, 0.006989731130391348, 0.006948208600585231, 0.00691130397433998, 0.006878484016187124, 0.006849280993363852, 0.006823283859637566, 0.006800130771295097, 0.006779502707804832, 0.006761118013004732  …  0.006761118013004732, 0.006779502707804832, 0.006800130771295097, 0.006823283859637566, 0.006849280993363852, 0.006878484016187124, 0.00691130397433998, 0.006948208600585231, 0.006989731130391348, 0.007036480732786138])

In [8]:
using StaticArrays

function SystemBathHamiltonian(ϵ₀::Float64, Γ::Float64, ϵ, γ)
    Hₛ = [ϵ₀]
    Hₗ = Diagonal(ApplyArray(vcat, ϵ, ϵ))
    
    aux = sqrt(Γ/(2π))
    κ =  @. aux*sqrt(γ)
    Hₛₗ = ApplyArray(vcat, κ, κ)

    H₁ = ApplyArray(hcat, Hₗ, Hₛₗ)
    H₂ = ApplyArray(hcat, Hₛₗ', Hₛ)

    return ApplyArray(vcat, H₁, H₂)
end

@btime $SystemBathHamiltonian($ϵ₀, $0.3, $ϵ, $γ)

  291.896 μs (270 allocations: 26.20 KiB)


vcat(hcat(400×400 Diagonal{Float64, ApplyArray{Float64, 1, typeof(vcat), Tuple{ApplyArray{Float64, 1, typeof(vcat), Tuple{Vector{Float64}, StepRangeLen{Float64, Float64, Float64, Int64}, StepRangeLen{Float64, Float64, Float64, Int64}, Vector{Float64}}}, ApplyArray{Float64, 1, typeof(vcat), Tuple{Vector{Float64}, StepRangeLen{Float64, Float64, Float64, Int64}, StepRangeLen{Float64, Float64, Float64, Int64}, Vector{Float64}}}}}}, vcat(vcat(20-element Vector{Float64}, 80-element Fill{Float64}, 80-element Fill{Float64}, 20-element Vector{Float64}), vcat(20-element Vector{Float64}, 80-element Fill{Float64}, 80-element Fill{Float64}, 20-element Vector{Float64}))), hcat((vcat(vcat(20-element Vector{Float64}, 80-element Fill{Float64}, 80-element Fill{Float64}, 20-element Vector{Float64}), vcat(20-element Vector{Float64}, 80-element Fill{Float64}, 80-element Fill{Float64}, 20-element Vector{Float64})))', 1-element Vector{Float64})):
 -1.0          ⋅           ⋅         …   ⋅          ⋅         

In [9]:
function TunnelingRates(ΔV::Float64, Tₗ::Float64, Tᵣ::Float64, ϵ, γ)
    ρₗ = @. γ*FermiDirac(ϵ, ΔV/2, Tₗ)
    ρᵣ = @. γ*FermiDirac.(ϵ, -ΔV/2, Tᵣ)
    
    Γ₊ = ApplyArray(vcat, ρₗ, ρᵣ, zeros(SVector{1}))
    Γ₋ = ApplyArray(vcat, γ.-ρₗ, γ.-ρᵣ, zeros(SVector{1}))
    
    return Diagonal(Γ₊), Diagonal(Γ₋)
end

@btime $TunnelingRates($ΔV, $Tₗ, $Tᵣ, $ϵ, $γ)

  376.242 ns (4 allocations: 832 bytes)


([0.007035049322774523 0.0 … 0.0 0.0; 0.0 0.006987444848479941 … 0.0 0.0; … ; 0.0 0.0 … 1.4314100116143693e-6 0.0; 0.0 0.0 … 0.0 0.0], [1.4314100116145922e-6 0.0 … 0.0 0.0; 0.0 2.286281911407194e-6 … 0.0 0.0; … ; 0.0 0.0 … 0.007035049322774523 0.0; 0.0 0.0 … 0.0 0.0])

In [10]:
function Liouvillian(ϵ₀::Float64, Γ::Float64, ΔV::Float64, Tₗ::Float64, Tᵣ::Float64, ϵ, γ)
    H = SystemBathHamiltonian(ϵ₀, Γ, ϵ, γ)
    Γ₊, Γ₋ = TunnelingRates(ΔV, Tₗ, Tᵣ, ϵ, γ)
    Ω = (Γ₋ - Γ₊)/2
    
    A1 = @. H-im*Ω
    B1 = @. im*Γ₊
    L₁ = ApplyArray(hcat, A1, B1)
    
    A2 = @. im*Γ₋
    B2 = @. H+im*Ω
    L₂ = ApplyArray(hcat, A2,  B2)
    
    return ApplyArray(vcat, L₁, L₂)
end

@btime $Liouvillian($ϵ₀, $0.3, $ΔV, $Tₗ, $Tᵣ, $ϵ, $γ)
L = Liouvillian(ϵ₀, 0.3, ΔV, Tₗ, Tᵣ, ϵ, γ);

  491.640 μs (478 allocations: 107.67 KiB)


In [16]:
function RunMachine(ϵ₀::Float64, W::Float64, W°::Float64, Γ::Float64, ΔV::Float64, Tₗ::Float64, Tᵣ::Float64, L₁::Int64, L₂::Int64)
    ϵ, γ = BathSpectra(W, W°, L₁, L₂)
    L = Liouvillian(ϵ₀, Γ, ΔV, Tₗ, Tᵣ, ϵ, γ) 
    
    λ, V = eigen(materialize(L),permute=false,scale=false)
    V⁻¹ = inv(materialize(V))
    D = @. 0.5*(sign(imag(λ))+1)
    Corr = V*Diagonal(D)*V⁻¹

    aₖaₖ= diag(Corr)[1:L₁+L₂]
    aₖc = Corr[1:L₁+L₂, 1+2(L₁+L₂)]
    caₖ = Corr[1+2(L₁+L₂), 1:L₁+L₂]

    A = @. FermiDirac(ϵ, ΔV/2, Tₗ) - real(aₖaₖ)
    B = @. real(aₖc) + real(caₖ)

    Jₚ = sum(@. γ*A)
    Jₕ = sum(@. γ*ϵ*A - √(Γ/(8π))*(γ^1.5)*B)

    return Jₚ, Jₕ
end

@btime $RunMachine($ϵ₀, $W, $W°, $0.3, $ΔV, $Tₗ, $Tᵣ, $L₁, $L₂)
RunMachine(ϵ₀, W, W°, 0.3, ΔV, Tₗ, Tᵣ, L₁, L₂);

  928.373 ms (2357 allocations: 52.81 MiB)


In [17]:
Γ = LinRange(0.0, 0.5, 20)
@time J = RunMachine.(ϵ₀, W, W°, Γ, ΔV, Tₗ, Tᵣ, L₁, L₂)

 26.478585 seconds (302.35 k allocations: 1.044 GiB, 0.16% gc time, 0.23% compilation time: 34% of which was recompilation)


20-element Vector{Tuple{Float64, Float64}}:
 (-1.0038669397097408e-15, -1.0630325199104779e-15)
 (0.0023905428504706837, 0.0002735536801837423)
 (0.00449938756301998, 0.0004716446043701313)
 (0.006358987048342775, 0.0006108031041391848)
 (0.007998254732266485, 0.0007039536729431912)
 (0.009442751358762068, 0.0007611941616473556)
 (0.010714947708746972, 0.0007904042079545336)
 (0.011834535292685964, 0.0007977296877459762)
 (0.01281876998234769, 0.0007879790995452273)
 (0.013682832246708585, 0.0007649547203843834)
 (0.01444017629419209, 0.0007317264687439137)
 (0.015102831896343195, 0.000690845057530983)
 (0.015681630313892466, 0.000644489554684404)
 (0.016186349316919674, 0.0005945529127133627)
 (0.016625796940361197, 0.0005426799393336418)
 (0.01700786477606701, 0.0004902776805908054)
 (0.01733957707176076, 0.0004385162378775345)
 (0.017627149457788652, 0.00038833177016770246)
 (0.01787605937716395, 0.0003404368625906347)
 (0.01809112347077391, 0.00029533879563476936)

In [18]:
using CSV
A = hcat(collect.(J)...)
vcat(collect(Γ)', A)'

20×3 adjoint(::Matrix{Float64}) with eltype Float64:
 0.0        -1.00387e-15  -1.06303e-15
 0.0263158   0.00239054    0.000273554
 0.0526316   0.00449939    0.000471645
 0.0789474   0.00635899    0.000610803
 0.105263    0.00799825    0.000703954
 0.131579    0.00944275    0.000761194
 0.157895    0.0107149     0.000790404
 0.184211    0.0118345     0.00079773
 0.210526    0.0128188     0.000787979
 0.236842    0.0136828     0.000764955
 0.263158    0.0144402     0.000731726
 0.289474    0.0151028     0.000690845
 0.315789    0.0156816     0.00064449
 0.342105    0.0161863     0.000594553
 0.368421    0.0166258     0.00054268
 0.394737    0.0170079     0.000490278
 0.421053    0.0173396     0.000438516
 0.447368    0.0176271     0.000388332
 0.473684    0.0178761     0.000340437
 0.5         0.0180911     0.000295339

In [14]:
Table = CSV.Tables.table(vcat(collect(Γ)', A)')

Tables.MatrixTable{Adjoint{Any, Matrix{Any}}} with 20 rows, 3 columns, and schema:
 :Column1  Any
 :Column2  Any
 :Column3  Any

In [15]:
DataFrame(Table)

Row,Column1,Column2,Column3
,Any,Any,Any
1,0.0,[0.0 7.83776e-17 … -8.11559e-16 -1.79102e-15],[-0.0 -7.37234e-17 … -7.63367e-16 -1.79102e-15]
2,0.0263158,[1.14406e-7 1.25938e-7 … -8.61706e-8 -7.60337e-8],[1.431e-8 1.55655e-8 … -1.00767e-8 -8.80469e-9]
3,0.0526316,[2.23025e-7 2.458e-7 … -1.71873e-7 -1.51747e-7],[2.77869e-8 2.98716e-8 … -1.9072e-8 -1.64968e-8]
4,0.0789474,[3.26659e-7 3.60494e-7 … -2.57446e-7 -2.27377e-7],[4.03312e-8 4.27998e-8 … -2.75572e-8 -2.35742e-8]
5,0.105263,[4.26039e-7 4.70856e-7 … -3.43265e-7 -3.03193e-7],[5.18248e-8 5.42074e-8 … -3.60894e-8 -3.05173e-8]
6,0.131579,[5.21836e-7 5.77659e-7 … -4.29745e-7 -3.79495e-7],[6.21333e-8 6.3929e-8 … -4.52214e-8 -3.77968e-8]
7,0.157895,[6.14669e-7 6.81627e-7 … -5.17343e-7 -4.56617e-7],[7.11065e-8 7.17776e-8 … -5.55112e-8 -4.58819e-8]
8,0.184211,[7.05117e-7 7.83442e-7 … -6.06561e-7 -5.34924e-7],[7.85791e-8 7.75455e-8 … -6.75319e-8 -5.52472e-8]
9,0.210526,[7.93722e-7 8.83754e-7 … -6.97954e-7 -6.14821e-7],[8.43712e-8 8.10039e-8 … -8.1881e-8 -6.63802e-8]
